In [ ]:
#O GeoPandas é um projeto open source que foi criado para tornar a manipulação de dados geoespaciais no Python uma tarefa mais simples. Ele estende as estruturas de dados do Pandas, que são o dataframe e o series, possibilitando a manipulação e tratamento de arquivos dessa natureza.
import geopandas as gpd
#O NumPy é uma poderosa biblioteca Python que é usada principalmente para realizar cálculos em Arrays Multidimensionais
import numpy as np 
#pandas é uma biblioteca de software criada para a linguagem Python para manipulação e análise de dados. Em particular, oferece estruturas e operações para manipular tabelas numéricas e séries temporais.
import pandas as pd 
#O módulo datetime fornece as classes para manipulação de datas e horas. Ainda que a aritmética de data e hora seja suportada, o foco da implementação é na extração eficiente do atributo para formatação da saída e manipulação. Funções gerais relacionadas ao calendário.
import datetime as dt
#O Matplotlib é uma biblioteca para a visualização de dados em Python. Ele apresenta uma API orientada a objetos que permite a criação de gráficos em 2D de uma forma simples e com poucos comandos
import matplotlib.pyplot as plt

#contém funções que podem criar figuras inteiras de uma vez
import plotly.express as px
#O plotly.figure_factorymódulo contém funções dedicadas à criação de tipos muito específicos de plotagens que eram difíceis de criar com objetos gráficos no momento de sua criação e antes da existência do Plotly Express 
import plotly.figure_factory as ff

In [ ]:
#criando a função "covid_plotl",
def covid_plot(df, abs = True, Today=True, date=None):
    '''
        grafico mundial de casos confirmados de covid-19 e morte.
        df = string, precisa ser 'confirmed_cases'(confirmados_casos) ou 'deaths'(mortes)
        abs = Boolean, o padrão é True. Os dados devem ser relatados em valor absoluto ou relativo? O valor relativo é calculado com base na população do país dividida por um milhão.
        Hoje = Booleano, o padrão é Verdadeiro. É um parâmetro para especificar se os dados são os dados de hoje, se definido igual a falso, uma data específica deve ser especificada
        date = dt.datetime, caso Hoje = False, é possível especificar uma data específica para o plot. Ele precisa estar no formato dt.datetime, então certifique-se
        para inserir o formato data = dt.datetime (AAAA, M, DD). Os dados começam em 2020,1,23, então certifique-se de selecionar uma data após esta.
    '''
    #verifica se o valor de "df" é diferente "deaths" ou "confirmed_cases", se for retorna que as opção disponiveis são "deaths" ou "confimed_cases"
    if df != 'deaths' and df != 'confirmed_cases':
        print('available option for df are: "deaths" or "confirmed_cases"')
    
    #verifica se o valor de "df" é "deaths" ou "confirmed_cases"
    if df == 'deaths' or df == 'confirmed_cases':
        #define a data inicial  como "23-01-2020"
        initial_data=dt.datetime(2020,1,23)
        
        #cria uma variavel com os dados de covid nos estados unidos
        us = pd.read_csv('../input/covid19-data-from-john-hopkins-university/CONVENIENT_us_{}.csv'.format(df),low_memory=False).iloc[1:,1:]
        #faz a soma dos valores do mesmo estado
        us = np.sum(us.astype(float),axis=1)
        #renomeia o conjunto para "United States of America"
        us = us.rename('United States of America')
        
        #define a variavel data com o dia atual
        data = dt.date.today().strftime("%d/%m/%Y")
        #cria a variavel "death" e popula ela com dados globais do covid
        death = pd.read_csv('../input/covid19-data-from-john-hopkins-university/CONVENIENT_global_{}.csv'.format(df))
        #cria o mapa padrão do geopandas
        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
        
        #cria um novo frame do pandas,vazio
        df_new=pd.DataFrame()
        #cria a variaves "states", como os valores abaixo.
        states=['Australia','Canada','China','Denmark','France','Netherlands','United Kindom']
        
        #faz um for, para cada registro em "states", executa uma ação
        for i in states:
            #para cada registro encontrado, ele organiza na mesma desde que seja as que estão no "states"
            colonna = ([col for col in death.columns if i in col])
            #adiciona a nova coluna na matriz df_new (0 colunas, 1 coluna, 2 colunas...)
            df_new =  pd.concat([df_new,(np.sum(death[colonna].iloc[1:].astype(float),axis=1))],axis=1)
            #deleta a coluna que foi usada da matriz de mortes
            death = death.drop(columns=colonna)
        #renomeia as colunas da matriz de acordo com os valores de "states"
        df_new.columns = states
        
        #deleta a colunas "Country/Region"
        death = death.drop(columns=['Country/Region'])
        #concatenas as a matriz  "df_new" com a "death"
        death = pd.concat([death,df_new],axis=1)
        #deixa a matriz sendo cada linha dados referente a um local, antes os locais eram por colunas
        death = death.T 
        #exclui a primeira coluna da matriz.
        death = death.drop(columns=[0])
        
        #adiciona os valores de (us) ao fim da matriz (death)
        death = death.append(us)
        #define o array(state) com os nomes das provincias e paises
        state = death.index
        #remove os indices escritos da matriz
        death.index = range(0,len(death))
        #salva o que antes eram os indices como os nomes dos conjuntos de dados
        death['name']= state
        #verifica se o parametro 'today' é verdadeiro e não tem data informada
        if Today==True and date==None:
            #cria a variavel test com que é o merge das matriz "world" e death
            test = pd.merge(world,death.iloc[:,-2:],on=['name'],how='left')
            test.iloc[:,-1] = test.iloc[:,-1].apply(lambda x: float(x))
            
            #verifica se a variavel "abs" está False
            if abs==False:
                #pega uma linha da matriz deixa ela indexavel (pega somente os valores daquela linha)
                population = test.iloc[:,0]
                #divide os valores por um milhão para exibir como relativo
                population_mil=population//1000000
                test.iloc[:,-1] = test.iloc[:,-1] //population_mil
        
        #verifica se o arg today é verdadeiro e o date é vazio
        if Today==True and date!=None:
            #se a condição for verdadeira printa "escolha apenas uma data"
            print('Choose only a date!')
    
        #verifica se o arg today é falso
        if Today==False:
            #verifica se date é vazio
            if date==None:
                #caso a condição seja verdadeira, imprime "entre com uma data especifica ou selecione today como verdadeiro para setar a date como hoje"
                 print('Either specifying a date or select Today = True to set date for today.')
        
        #verifica se date é diferente de vazio
        if date!=None:
            #verifica se o tipo de date é diferente de datetime
            if type(date) != dt.datetime:
                #se a condição for verdadeira, pede pra data ser inserida conforme o padrão.
                print('be sure that date format is: "dt.datetime(YYYY,M,DD)"')
            
            #verifica se o date é igual ao datetime
            if type(date) == dt.datetime:
                #se for, colocar o valor de date na variavel data
                data = date
                #define a variavel index os dias entre a data inicial do primeiro registro e a data iformada
                index=((data-initial_data).days)
                if index < 0:
                    #se o index for menor que 0, informa que a primeira data registrada é 23-1-2020
                    print('data are recorded starting from 2020,1,23; be sure to select a value after this date')
                if index >= 0:
                    #caso contrario, transforma a data em um string
                    data = (data.strftime("%d/%m/%Y"))
                    # faz a união dos registros de world e death em que o death.name= world.name
                    test = pd.merge(world,death[[index,'name']],on=['name'],how='left')
                    test.iloc[:,-1] = test.iloc[:,-1].apply(lambda x: float(x))
                
            #verifica se o valor absoluto é falso
            if abs==False:
                #se for absoluto pega os valores da linha 0
                population = test.iloc[:,0]
                # e divide eles por um milhão
                population_mil=population//1000000
                test.iloc[:,-1] = test.iloc[:,-1] //population_mil

        #essa parte não entendi direito o motivo de ter limitado os valores maximos, no caso de relativo ok, mas absoluto não mas ok..
        #verifica se df é "confirmed_cases"
        if  df ==  'confirmed_cases':
            #Define o valor maximo como duzentos mil
            maxs=200000
            #Se o valor absoluto for false mostra somente um mil no maximo
            if abs==False:
                maxs=1000
        #verifica se é df é deaths
        if  df == 'deaths':
            #se for ja define o valor maximo como 3500
            maxs=3500
            if abs==False:
                #se o valor absoluto for false, mostra no maximo 35
                maxs=35

        ab = 'Absolute'
        if abs==False:
            ab= 'Relative'
            
        #define as dimensões da imagem
        fig,ax=plt.subplots(figsize=(25,25))
        #define os dados da imagem, informações a serem preenchidas, valores maximo, valores minimo, legenda
        #rotulos, orientação,e cores para ausencia de dados
        
        #define o valor minimo como  0, e o valor maximo conforme variavel definida anteriormente.
        # e habilita a legenda.
        test.plot(column=test.iloc[:,-1],vmin=0,vmax=maxs,legend=True, 
              #define a legenda como posição horizontal, e o texto "number of deths" ou "number of confirmed_cases"
              legend_kwds={
        'label': "number of {}".format(df),
        'orientation': "horizontal"},
              #para valores ausentes, ficará pintado de cinza claro
              missing_kwds={
                'color':'lightgrey',
                'label':'Missing values'
              } , ax=ax
             )
        #define o titulo como "abs world covid-19 df in data", exemplo "Absolute world covid-19 deaths in 20/11/2020"
        ax.set_title("{} world covid-19 {} in {}".format(ab,df,data), fontsize=25)


In [ ]:
#imprime um gráfico, com valores do dia 12/02/2020, e valor absoluto referente a mortes
covid_plot('deaths',abs=True,Today=False,date=dt.datetime(2020,2,12))

In [ ]:
#imprime um gráfico pintado os valores relativos de mortes no dia atual
covid_plot('deaths',abs=False)

In [ ]:

covid_plot('confirmed_cases')

In [ ]:
covid_plot('confirmed_cases',abs=False)